<a href="https://colab.research.google.com/github/dam-crtl/GCN/blob/main/gnn_for_bitcoin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
print(torch.__version__)

2.0.0+cu117


In [3]:
!pip install torch==2.0.0 torchvision==0.15.1 torchaudio==2.0.1
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html
!pip install torch-geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv

In [6]:
# データセットのロード（Cora）
dataset = Planetoid(root='/path/to/dataset', name='Cora')

Processing...
Done!


In [7]:
# データの前処理
data = dataset[0]  # 最初のグラフデータを取得
x = data.x  # ノードの特徴量
y = data.y  # ノードのクラスラベル
edge_index = data.edge_index  # エッジのインデックス

In [8]:
# GNNモデルの定義
class GNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

In [9]:
# モデルの初期化
input_dim = dataset.num_features
hidden_dim = 16
output_dim = dataset.num_classes
model = GNN(input_dim, hidden_dim, output_dim)

In [11]:
# 損失関数と最適化手法の定義
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [12]:
# モデルの学習
model.train()
optimizer.zero_grad()
output = model(x, edge_index)
loss = criterion(output[data.train_mask], y[data.train_mask])
loss.backward()
optimizer.step()

In [13]:
# 推論
model.eval()
output = model(x, edge_index)
predicted_labels = output.argmax(dim=1)

In [14]:
print("Predicted labels:", predicted_labels)

Predicted labels: tensor([6, 4, 4,  ..., 6, 6, 6])
